In [8]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 5
EPOCHS = 40
POSITION = "lying_down_left"
MODEL_NAME = f"position:{POSITION}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 5
TEST_SIZE = 8
LEAVE_ONE_OUT = False
NORMALISE = True
RANDOM_TEST = False
TEST_DEV_OVERLAP = True



### This code takes in a position, eg "sitting_or_standing", "lying_down_left" etc and trains a model for just classifying ONLY RESPIRATORY activities of that position 

In [9]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split
import split_by_student
import time


POSITIONS = [
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
]

STANDING_OR_SITTING_ACTIVITIES = [
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",

]

STANDING_OR_SITTING_ACTIVITIES_COMBINED = [
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",
    "sitting_or_standing&normal_breathing",

]


LYING_DOWN_LEFT_ACTIVITIES = [
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&normal_breathing"

]

LYING_DOWN_RIGHT_ACTIVITIES = [
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_rightbreathing"
]

LYING_DOWN_BACK_ACTIVITIES = [
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&normal_breathing"
]

LYING_DOWN_STOMACH_ACTIVITIES = [
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&normal_breathing"
]

DATA_DIRECTORY = "./all_respeck"

position_activities = {"sitting_or_standing": STANDING_OR_SITTING_ACTIVITIES,
                       "lying_down_left": LYING_DOWN_LEFT_ACTIVITIES,
                       "lying_down_right": LYING_DOWN_RIGHT_ACTIVITIES,
                       "lying_down_back": LYING_DOWN_BACK_ACTIVITIES,
                       "lying_down_stomach": LYING_DOWN_STOMACH_ACTIVITIES}

POSSIBLE_ACTIVITIES = position_activities[POSITION]

if POSITION == "sitting_or_standing":
    POSSIBLE_OUTCOMES = STANDING_OR_SITTING_ACTIVITIES_COMBINED
else:
    POSSIBLE_OUTCOMES = POSSIBLE_ACTIVITIES


LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}

if TEST_DEV_OVERLAP:
    TEST_OVERLAP_SIZE = 0
else:
    TEST_OVERLAP_SIZE = SEQUENCE_OVERLAP

In [10]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in POSSIBLE_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if csv_file in file_names:
                if gyro:
                    sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)
                else:
                    sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap, normalise=NORMALISE)

                # iterate through each generated sequence
                for sequence in sequences:
                    position = key.split("&")[0]
                    activity = key.split("&")[1]
                    if position == "standing" or position == "sitting":
                        tagged_data.append(("sitting_or_standing" + "&" + activity, sequence))
                    else:
                        tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [11]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        #layers.Dropout(0.5),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        #layers.Conv1D(512, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        #layers.Dense(256, activation='relu'),
        #layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [12]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size, randomise=RANDOM_TEST)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_OVERLAP_SIZE, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, TEST_OVERLAP_SIZE, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [13]:
def leave_one_out():
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([], [])) #batch_size, epochs
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))
        print("number of students tested so far: " + str(len(test_accuracies)))
        time.sleep(3)
        
        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))


In [14]:
if LEAVE_ONE_OUT:
    leave_one_out()

else:

    train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)


    # train and save model (CHOOSE BETWEEN CNN AND LSTM)
    model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs

    print("labels by index:")
    for label, index in LABEL_TO_INDEX.items():
        print(f"{index}: {label}")

    # Evaluate the model on the test set
    if TEST_SIZE >0:
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

    # Save the trained model
    #model.save(f"models/Task2/{MODEL_NAME}_{test_accuracy}.keras")

Train Set: s80, s15, s64, s36, s21, s17, s98, s61, s45, s83, s46, s88, s59, s29, s86, s18, s63, s33, s1, s35, s67, s42, s71, s57, s39, s72, s48, s75, s52, s66, s55, s13, s50, s84, s11, s30, s9, s56, s5, s3, s44, s92, s43, s82, s22, s34, s65, s93, s54, s8, s40, s7, s97, s74, s87, s77, s16, s32
Dev Set: s70, s91, s51, s38, s95
Test Set: s12, s96, s79, s27, s23, s102, s60, s100
there are 4683 tagged sequences in the dataset
there are 91 tagged sequences in the dataset
there are 168 tagged sequences in the dataset
Epoch 1/40
937/937 [==============================] - 3s 3ms/step - loss: 0.9948 - accuracy: 0.4463 - val_loss: 1.1646 - val_accuracy: 0.6044
Epoch 2/40
937/937 [==============================] - 3s 3ms/step - loss: 0.5857 - accuracy: 0.7322 - val_loss: 0.7012 - val_accuracy: 0.7253
Epoch 3/40
937/937 [==============================] - 2s 3ms/step - loss: 0.4711 - accuracy: 0.7954 - val_loss: 0.6867 - val_accuracy: 0.7582
Epoch 4/40
937/937 [==============================] - 2s 3